# Pebblo 安全文档加载器

> [Pebblo](https://daxa-ai.github.io/pebblo/) 使开发人员能够安全地加载数据，并将他们的 Gen AI 应用推广到生产环境，而无需担心组织的合规性和安全要求。该项目识别加载数据中存在的语义主题和实体，并在 UI 或 PDF 报告中对其进行总结。

Pebblo 有两个组件。

1. 适用于 Langchain 的 Pebblo 安全文档加载器
1. Pebblo 服务器

本文档将介绍如何使用 Pebblo 安全文档加载器增强您现有的 Langchain 文档加载器，以深入了解摄入 Gen-AI Langchain 应用程序中的主题和实体类型。有关 `Pebblo Server` 的详细信息，请参阅此 [pebblo server](https://daxa-ai.github.io/pebblo/daemon) 文档。

Pebblo Safeloader 为 Langchain `DocumentLoader` 实现了安全的数据摄入。这是通过使用 `Pebblo Safe DocumentLoader` 包装文档加载器调用来实现的。

注意：要在 Pebblo 默认的 (localhost:8000) URL 以外的其他 URL 上配置 pebblo 服务器，请将正确的 URL 放在 `PEBBLO_CLASSIFIER_URL` 环境变量中。这也可以使用 `classifier_url` 关键字参数进行配置。参考：[server-configurations](https://daxa-ai.github.io/pebblo/config)

#### 如何启用 Pebblo 文档加载？

假设一个 Langchain RAG 应用程序片段使用 `CSVLoader` 来读取 CSV 文档进行推理。

以下是使用 `CSVLoader` 进行文档加载的代码片段。

In [ ]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("data/corp_sens_data.csv")
documents = loader.load()
print(documents)

Pebblo SafeLoader 可以在上述代码片段中通过几行代码进行更改来启用。

In [ ]:
from langchain_community.document_loaders import CSVLoader, PebbloSafeLoader

loader = PebbloSafeLoader(
    CSVLoader("data/corp_sens_data.csv"),
    name="acme-corp-rag-1",  # App name (Mandatory)
    owner="Joe Smith",  # Owner (Optional)
    description="Support productivity RAG application",  # Description (Optional)
)
documents = loader.load()
print(documents)

### 将语义主题和身份信息发送到 Pebblo 云服务器

要将语义数据发送到 pebblo-cloud，请将 `api-key` 作为参数传递给 `PebbloSafeLoader`，或者将 `api-key` 设置在 `PEBBLO_API_KEY` 环境变量中。

In [ ]:
from langchain_community.document_loaders import CSVLoader, PebbloSafeLoader

loader = PebbloSafeLoader(
    CSVLoader("data/corp_sens_data.csv"),
    name="acme-corp-rag-1",  # App name (Mandatory)
    owner="Joe Smith",  # Owner (Optional)
    description="Support productivity RAG application",  # Description (Optional)
    api_key="my-api-key",  # API key (Optional, can be set in the environment variable PEBBLO_API_KEY)
)
documents = loader.load()
print(documents)

### 为加载的元数据添加语义主题和身份

要为加载文档的元数据添加语义主题和语义实体，请将 `load_semantic` 设置为 `True` 作为参数，或者定义一个新的环境变量 `PEBBLO_LOAD_SEMANTIC` 并将其设置为 `True`。

In [ ]:
from langchain_community.document_loaders import CSVLoader, PebbloSafeLoader

loader = PebbloSafeLoader(
    CSVLoader("data/corp_sens_data.csv"),
    name="acme-corp-rag-1",  # App name (Mandatory)
    owner="Joe Smith",  # Owner (Optional)
    description="Support productivity RAG application",  # Description (Optional)
    api_key="my-api-key",  # API key (Optional, can be set in the environment variable PEBBLO_API_KEY)
    load_semantic=True,  # Load semantic data (Optional, default is False, can be set in the environment variable PEBBLO_LOAD_SEMANTIC)
)
documents = loader.load()
print(documents[0].metadata)

### 匿名化代码片段以隐藏所有个人身份信息

将 `anonymize_snippets` 设置为 `True`，以匿名化进入 VectorDB 的代码片段以及生成的报告中的所有个人身份信息 (PII)。

> 注意：_Pebblo Entity Classifier_ 可有效识别个人身份信息 (PII)，并且在持续进步中。虽然其召回率尚未达到 100%，但正在稳步提高。
> 如需了解更多详情，请参阅[_Pebblo Entity Classifier 文档_](https://daxa-ai.github.io/pebblo/entityclassifier/)

In [ ]:
from langchain_community.document_loaders import CSVLoader, PebbloSafeLoader

loader = PebbloSafeLoader(
    CSVLoader("data/corp_sens_data.csv"),
    name="acme-corp-rag-1",  # App name (Mandatory)
    owner="Joe Smith",  # Owner (Optional)
    description="Support productivity RAG application",  # Description (Optional)
    anonymize_snippets=True,  # Whether to anonymize entities in the PDF Report (Optional, default=False)
)
documents = loader.load()
print(documents[0].metadata)